In [4]:
import robotic as ry
import numpy as np
import random
import time
from bridgebuild_env import create_n_boxes
# this import the local manipulation.py .. to be integrated in robotic..
import manipulation as manip

### Task Env: Bridge Builder

In [5]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
n_boxes = 3
keep_dist_between = ["box1", "box2", "box3"]
C = create_n_boxes(C = C, N = n_boxes, position = 'fixed')

# remove so collision with wrist camera is ignored
C.delFrame("panda_collCameraWrist")

# for convenience, a few definitions:
qHome = C.getJointState()
gripper = "l_gripper";
palm = "l_palm";
box1 = "box1";
box2 = "box2";
box3 = "box3";
table = "table";
boxSize = C.frame(box1).getSize()

C.view()

0

In [6]:
C.setJointState(qHome)
C.view_raise()

for l in range(100):
        box = np.random.choice([box1, box2, box3])
        qStart = C.getJointState()

        graspDirection = 'x' #random.choice(['y', 'z']) #'x' not possible: box too large
        placeDirection = 'z' #random.choice(['x', 'y', 'z', 'xNeg', 'yNeg', 'zNeg'])
        info = f'placement {l}: grasp {graspDirection} place {placeDirection}'
        print('===', info)

        M = manip.ManipulationModelling(C, info, helpers=[gripper])
        M.setup_pick_and_place_waypoints(gripper, box, homing_scale=1e-1)
        M.grasp_box(1., gripper, box, palm, graspDirection)
        print('place direction:', placeDirection)
        M.place_box(2., box, table, palm, placeDirection)
        M.no_collision([], palm, table)
        M.auto_pairwise_keep_dist([], keep_dist_between, margin = 0.07)
        # x, y, z, smooth - table size: array([2.5 , 2.5 , 0.1 , 0.02])
        # place inital block as chosen by np.random.choice at a random location
        M.target_relative_xy_position(2., gripper, table, [np.random.uniform(0.0, 0.5), np.random.uniform(0.0, 0.5)])
        ways = M.solve()

        if not M.feasible:
            continue

        M2 = M.sub_motion(0)
        # M = manip.ManipulationModelling(C, info, helpers=[gripper])
        # M.setup_point_to_point_motion(qStart, ways[0])
        M2.no_collision([.3,.7], palm, box)
        
        M2.retract([.0, .2], gripper, dist = .13)
        M2.approach([.8, 1.], gripper)
        M2.solve()
        if not M2.feasible:
            continue

        M3 = M.sub_motion(1)
        #manip.ManipulationModelling(C, info)
        # M.setup_point_to_point_motion(ways[0], ways[1])
        M3.no_collision([], table, box)

        M3.solve()
        if not M3.ret.feasible:
            continue

        M2.play(C)
        C.attach(gripper, box)
        M3.play(C)
        C.attach(table, box)

del M

=== placement 0: grasp x place z
place direction: z
  -- feasible:placement 0: grasp x place z
     { time: 0.02899, evals: 32, done: 1, feasible: 1, sos: 0.115417, f: 0, ineq: 3.13974e-06, eq: 0.0267142 }
  -- feasible:sub_motion_0--placement 0: grasp x place z
     { time: 0.124609, evals: 20, done: 1, feasible: 1, sos: 3.17957, f: 0, ineq: 2.65196e-07, eq: 0.475916 }
  -- feasible:sub_motion_1--placement 0: grasp x place z
     { time: 1.16362, evals: 183, done: 1, feasible: 1, sos: 0.209885, f: 0, ineq: 0.00120969, eq: 0.32483 }
=== placement 1: grasp x place z
place direction: z
  -- feasible:placement 1: grasp x place z
     { time: 0.051027, evals: 27, done: 1, feasible: 1, sos: 0.0869552, f: 0, ineq: 1.02995e-05, eq: 0.0162298 }
  -- feasible:sub_motion_0--placement 1: grasp x place z
     { time: 0.150675, evals: 26, done: 1, feasible: 1, sos: 1.74471, f: 0, ineq: 3.58791e-07, eq: 0.317771 }
  -- feasible:sub_motion_1--placement 1: grasp x place z
     { time: 0.503505, evals: